# A Real-world Example Using HBV-SASK Hydrological Model

## Installation (from source)

The latest version of the varstool sensitivity and uncertainty package can always be retrieved from its GitHub repo using the following commands:

In [1]:
!git clone https://github.com/vars-tool/vars-tool
!pip install vars-tool/.

Cloning into 'vars-tool'...


Processing c:\kasra\vars-tool
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for varstool: filename=varstool-2.1-py2.py3-none-any.whl size=430049 sha256=a17adb8156d5191da41e846edb8ffd44a3cbee1b386a2538eeb704e1e7c5407b
  Stored in directory: c:\users\kak136\appdata\local\pip\cache\wheels\03\b7\01\90265a9e06073dc5d3f061ea0cf6edb06870c2af0a14841906
Successfully built varstool
  Attempting uninstall: varstool
    Found existing installation: varstool 2.1
    Uninstalling varstool-2.1:
      Successfully uninstalled varstool-2.1


  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


## HBV-SASK Model - A Real-world Sensitivity Analysis Problem

The HBV-SASK Model is a real-world hydrological model that has been developed within the [**"Watershed Systems Analysis and Modelling Lab"**](https://www.samanrazavi.com) for educational purposes. In this example, we analyze the sensitivity of each parameter of the HBV-SASK model to the simulated streamflow (i.e., output) using the state-of-the-art varstool Python package.

HBV-SASK utilizes daily time-series of precipitation and temperature and produced many outputs, such as streamflow, evapotransipration, etc.

For more information see: Razavi, S., Sheikholeslami, R., Gupta, H. V., & Haghnegahdar, A. (2019). ***VARS-TOOL: A toolbox for comprehensive, efficient, and robust sensitivity and uncertainty analysis.*** Environmental modelling & software, 112, 95-107. doi: https://doi.org/10.1016/j.envsoft.2018.10.005

In [2]:
from varstool import TSVARS, Model

# Loading the "HBV_SASK" model that is included in this package
from varstool.funcs import HBV_SASK

exp_1_model = Model(HBV_SASK)

Let's just see how the model works. The object "Model" is an standard way of wrapping different functions that are going to be used by varstool. Therefore, users must communicate with their function of interest using this simple wrapper class. The arguments of this class can be quickly viewed by the following command:

In [3]:
# Model?

To quickly test the HBV-SASK model with some random numbers as its input, follow the cell below:

In [4]:
import pandas as pd

params={#name  #value
       'TT'   : -4.00,
       'C0'   : 0.00,
       'ETF'  : 0.00,
       'LP'   : 0.00,
       'FC'   : 50.0,
       'beta' : 1.00,
       'FRAC' : 0.10,
       'K1'   : 0.05,
       'alpha': 1.00,
       'K2'   : 0.00,
       'UBAS' : 1.00,
       'PM'   : 0.50,
}

exp_1_model(pd.Series(params))

C:\Kasra\ProgramFiles\lib\site-packages\varstool\funcs\hbv.py:180: RuntimeWarning: invalid value encountered in double_scalars
  AET = PET * ( SMS / LP )


(               Q_cms      Q_mm  AET  PET        Q1  Q1routed   Q2  ponding
 1950-01-01  3.782170  0.150000  0.0  0.0  0.150000  0.150000  0.0    0.000
 1950-01-02  3.593062  0.142500  0.0  0.0  0.142500  0.142500  0.0    0.000
 1950-01-03  3.413409  0.135375  0.0  0.0  0.135375  0.135375  0.0    0.000
 1950-01-04  3.242738  0.128606  0.0  0.0  0.128606  0.128606  0.0    0.000
 1950-01-05  3.080601  0.122176  0.0  0.0  0.122176  0.122176  0.0    0.000
 ...              ...       ...  ...  ...       ...       ...  ...      ...
 2011-12-27  0.798471  0.031667  NaN  0.0  0.031667  0.031667  0.0    0.000
 2011-12-28  0.758548  0.030084  NaN  0.0  0.030084  0.030084  0.0    4.435
 2011-12-29  1.279751  0.050755  NaN  0.0  0.050755  0.050755  0.0    2.255
 2011-12-30  1.500057  0.059492  NaN  0.0  0.059492  0.059492  0.0    0.000
 2011-12-31  1.425054  0.056517  NaN  0.0  0.056517  0.056517  0.0    0.000
 
 [22645 rows x 8 columns],
                  SWE  SMS        S1            S2
 1950-01

HBV-SASK returns many outputs; for the sake of simplicity, here we just use the simulated streamflow as the chosen model response (shown below). As HBV-SASK returns time-series of streamflow, the time-series version of the **`VARS`** algorithm is employed. The time-series version of this algorithm is detailed in the following publication:

* Gupta, H. V., & Razavi, S. (2018). Revisiting the basis of sensitivity analysis for dynamical earth system models. Water Resources Research, 54(11), 8692-8717. doi: https://doi.org/10.1029/2018WR022668

In [5]:
# custome output selection - we are just now using the last 20 time-steps
def custom_HBV_SASK(x):
    # on MS Windows machines, import the original library again in
    # the body of the function to be wrapped, if any.
    # No need to import again on *nix machines
    from varstool.funcs import HBV_SASK
    
    return HBV_SASK(x)[0]['Q_cms'][-20:]

exp_1_model = Model(custom_HBV_SASK)

# and testing the output
exp_1_model(pd.Series(params))

C:\Kasra\ProgramFiles\lib\site-packages\varstool\funcs\hbv.py:180: RuntimeWarning: invalid value encountered in double_scalars
  AET = PET * ( SMS / LP )


2011-12-12    0.758757
2011-12-13    0.720819
2011-12-14    0.684778
2011-12-15    0.650539
2011-12-16    0.618012
2011-12-17    0.587112
2011-12-18    1.266913
2011-12-19    1.203567
2011-12-20    1.143389
2011-12-21    1.086220
2011-12-22    1.031909
2011-12-23    0.980313
2011-12-24    0.931298
2011-12-25    0.884733
2011-12-26    0.840496
2011-12-27    0.798471
2011-12-28    0.758548
2011-12-29    1.279751
2011-12-30    1.500057
2011-12-31    1.425054
Name: Q_cms, dtype: float64

In [6]:
# setting up a TSVARS experiment for the HBV-SASK model

exp_1 = TSVARS(num_stars=2,
               parameters={  #lb     #ub
                   'TT'   : [-4.00, 4.00],
                   'C0'   : [0.00 , 10.0],
                   'ETF'  : [0.00 , 1.00],
                   'LP'   : [0.00 , 1.00],
                   'FC'   : [50.0 ,500.0],
                   'beta' : [1.00 , 3.00],
                   'FRAC' : [0.10 , 0.90],
                   'K1'   : [0.05 , 1.00],
                   'alpha': [1.00 , 3.00],
                   'K2'   : [0.00 , 0.05],
                   'UBAS' : [1.00 , 3.00],
                   'PM'   : [0.50 , 2.00],
               },
               delta_h=0.1,
               ivars_scales=(0.1, 0.3, 0.5),
               model=exp_1_model,
               seed=1001,
               sampler='lhs',
               bootstrap_flag=False,
               grouping_flag=False,
               report_verbose=True,
               func_eval_method='parallel',
               vars_eval_method='parallel',
               vars_chunk_size=None,
          )

At any time, the status of the VARS analysis could be viewed by typing the name of the instance that is used to initiate the experiment. Here, it is `exp_1`:

In [7]:
exp_1

Star Centres: Loaded
Star Points: Not Loaded
Parameters: 12 paremeters set
Delta h: 0.1
Model: custom_HBV_SASK
Seed Number: 1001
Bootstrap: Off
Bootstrap Size: N/A
Bootstrap CI: N/A
Grouping: Off
Number of Groups: None
Function Evaluation Method: parallel
TSVARS Evaluation Method: parallel
TSVARS Chunk Size: N/A
Verbose: On
TSVARS Analysis: Not Done

From the status report, it could be viewed that the `star_centres` are fully sampled (via `lhs`) and loaded. Meanwhile, $\Delta$h (`delta_h`) is set to 0.1, `seed` is set to 1001, and 12 parameters are fully set to begin the VARS sensitivity analysis.

Time-series VARS (TSVARS) comes with two computational methods to carry out sensitivity analysis, that are:
1. Serial and
2. Parallel.

It is worth noting that these computational methods are applicable to: **a**) while varstool runs the `Model` and **b**) when varstool computes common `VARS` indices. For the former `func_eval_method` method must be changed and for the latter `vars_eval_method` to either `'serial'` or `'parallel'`.

Furthermore, to save memory usage, the computations can be chunked into several segments to avoid memory leak. In this example, first a simple serial version without chunking is demonstrated. It should be noted that the parallel version are unstable and might result in unexpected errors.

Before carrying out a sensitivity analysis, let's check the loaded `star_centres`:

In [8]:
exp_1.star_centres

array([[0.74408722, 0.96041308, 0.09803003, 0.21526074, 0.01155678,
        0.09789096, 0.9911775 , 0.51015798, 0.85274302, 0.29022856,
        0.6763727 , 0.02056527],
       [0.15311609, 0.13253178, 0.55455094, 0.70552831, 0.73506341,
        0.56364827, 0.17640264, 0.11162101, 0.30676093, 0.98275457,
        0.42678384, 0.80294738]])

It could be seen that 2 star centres for 12 parameters of the HBV-SASK model are successfully produced. Now let's move on to the sensitivity analysis bit:

In [ ]:
exp_1.run_online()

C:\Kasra\ProgramFiles\lib\site-packages\varstool\vars_sa.py:1096: UserWarning: Evaluating function in parallel mode is not stable yet. varstool currently uses `mapply` to parallelize function evaluations, see https://github.com/ddelange/mapply
  stacklevel=1


  0%|                                                                                          | 0/240 [00:00<…

You can see the list of chosen outputs in the `output` attribute of the VARS analysis experiment:

In [ ]:
exp_1.output.keys()

To view the produced variogram:

In [ ]:
exp_1.output['Gamma'].unstack(level=1) # unstack to make the dataframe more legible

# DYI

How about you test other outputs of HBV-SASK yourself below?

In [ ]:
exp_2_model = Model(...)

exp_2 = TSVARS(...)